# Operaciones básicas

Trataremos aquí las operaciones más básicas que se pueden realizar sobre las estructuras de datos de pandas. Se separan en un notebook aparte porque estas operaciones tienen un funcionamiento prácticamente idéntico en Series y DataFrames. En caso de que esto no sea así en algún caso concreto se indicará explícitamente.

In [1]:
import numpy as np
import pandas as pd

## Tratamiento de Series y DataFrames como diccionarios

Dado que internamente tanto las Series como los DataFrames pueden verse como diccionarios, podemos apilcar sobre los mismos cualquier funcionalidad que aplicaríamos sobre diccionarios básicos del core de Python.<br/>
<b>IMPORTANTE:</b> Hay que tener en cuenta que en DataFrames el diccionario es un diccionario de "columnas".

In [2]:
serie = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [3]:
dataframe = pd.DataFrame({'var1': serie, 'var2': serie})
dataframe

,var1,var2
a,1,1
b,2,2
c,3,3
d,4,4


#### Indexación por clave

In [4]:
# Indexación mediante clave del índice en series
serie['a']

1

In [5]:
# Indexación por nombre de columna en dataframes
dataframe['var2']

a    1
b    2
c    3
d    4
Name: var2, dtype: int64

#### Comprobación de la existencia de una clave

In [6]:
# Comprobación de la existencia de una clave en el índice en series
'b' in serie

True

In [7]:
# Comprobación de la existencia de un nombre de columna en dataframes
'b' in dataframe

False

In [8]:
# Comprobación de la existencia de un nombre de columna en dataframes
'var1' in dataframe

True

#### Adición de elementos

<b>IMPORTANTE:</b> Al añadir columnas a un DataFrame, el tamaño del vector añadido deberá coincidir con el del DataFrame original. En caso contrario se recibirá un error.

In [9]:
# Adición de elementos a series
serie['e'] = 5
serie

a    1
b    2
c    3
d    4
e    5
dtype: int64

In [11]:
# Adición de elementos a datafrmaes
dataframe['var3'] = [5, 6, 7, 8]
dataframe

,var1,var2,var3
a,1,1,5
b,2,2,6
c,3,3,7
d,4,4,8


#### Eliminación de elementos

In [12]:
# Eliminación de elementos en series
del serie['e']
serie

a    1
b    2
c    3
d    4
dtype: int64

In [13]:
dataframe

,var1,var2,var3
a,1,1,5
b,2,2,6
c,3,3,7
d,4,4,8


In [14]:
# Eliminación de columnas en dataframes
del dataframe['var2']
dataframe

,var1,var3
a,1,5
b,2,6
c,3,7
d,4,8


## Tratamiento de Series y DataFrames como ndarrays

Dado que, internamente, cualquier estructura de pandas está implementada sobre ndarrays de NumPy, es posible realizar sobre Series y DataFrames todas las operaciones que se pueden realizar sobre un ndarrays.<br/>

<b>IMPORTANTE:</b> Dado que un ndarray no puede mezclar elementos de diferentes tipos y un DataFrame sí, algunas de las operaciones sobre DataFrames estarán supeditadas a que todas sus columnas tengan el mismo tipo.

In [15]:
serie = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [16]:
dataframe = pd.DataFrame({'var1': pd.Series(serie, dtype=np.int32), 'var2': pd.Series(serie, dtype=np.string_)})
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


#### Consulta de la composición

Disponemos de los mismos atributos de consulta que en ndarrays, si bien hay que tener en cuenta que:<br/>
<ul>
<li>El atributo <b>dtype</b> será <b>dtypes</b> en DataFrames dada la posibilidad de múltiples tipos.</li>
<li>El atributo <b>ndim</b> en Series siempre valdrá 1 dado que siempre son estructuras unidimensionales y 2 en DataFrames dado que siempre son estructuras bidimensionales.</li>
</ul>

In [17]:
# Consulta del tipo almacenado en una serie
serie.dtype

dtype('int64')

In [18]:
# Consulta de los tipos almacenados en un dataframe
dataframe.dtypes

var1     int32
var2    object
dtype: object

In [26]:
dataframe.dtypes[0]

dtype('int32')

In [19]:
# Consulta del número de dimensiones en una serie
serie.ndim

1

In [20]:
# Consulta del número de dimensiones en un dataframe
dataframe.ndim

2

In [21]:
# Consulta de la forma de una serie
serie.shape

(4,)

In [22]:
# Consulta de la forma de un dataframe
dataframe.shape

(4, 2)

In [25]:
dataframe.shape[0]

4

In [23]:
# Consulta del número de elementos de una serie
serie.size

4

In [24]:
# Consulta del número de elementos de un dataframe
dataframe.size

8

#### Operaciones con escalares

Al aplicar una operación sobre una estructura de pandas y un escalar se obtendrá otra estructura de pandas de idénticas características a la inicial pero con la operación aplicada elemento a elemento, <b>manteniendo el índice inalterado</b>.<br/>

<b>IMPORTANTE</b>: Dado que un DataFrame puede mezclar tipos muy diferentes en sus columnas, la aplicación de una operación con un escalar elemento a elemento puede no ser válida (p.e. operaciones matemáticas sobre cadenas).

In [28]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [29]:
# Suma de series y escalar
serie + 2

a    3
b    4
c    5
d    6
dtype: int64

In [30]:
# División de series y escalar
1 / serie

a    1.000000
b    0.500000
c    0.333333
d    0.250000
dtype: float64

In [31]:
# Multiplicación de dataframe y escalar
dataframe * 2

,var1,var2
a,2,b'11'
b,4,b'22'
c,6,b'33'
d,8,b'44'


In [32]:
# División de dataframe y escalar
1 / dataframe

TypeError: unsupported operand type(s) for /: 'int' and 'bytes'

#### Operaciones entre estructuras de pandas

Al aplicar una operación entre estructuras de pandas se aplicará la misma elemento a elemento. En el caso de pandas no es necesario, como lo era en NumPy, que los operandos tengan el mismo tamaño y forma ya que se aplicará un proceso de "alineación". Este proceso devolverá:<br/>
<ul>
<li>Como índices: la unión de las claves de ambos operandos.</li>
<li>Como valores: el resultado de aplicar la operación entre cada pareja de elementos (si coinciden las claves entre ambos operandos) o NaN (en caso contrario).</li>
</ul>

<b>IMPORTANTE:</b> De nuevo, el hecho de que un DataFrame pueda mezclar tipos en sus contenidos hace que no todas las operaciones matemáticas se puedan aplicar a los mismos.

In [33]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [34]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [35]:
serie1 = serie.copy()
serie1['e'] = 7
serie1

a    1
b    2
c    3
d    4
e    7
dtype: int64

In [36]:
dataframe1 = dataframe.copy()
dataframe1['var3'] = [1, 2, 3, 4]
dataframe1

,var1,var2,var3
a,1,b'1',1
b,2,b'2',2
c,3,b'3',3
d,4,b'4',4


In [37]:
# Suma de series
serie + serie1

a    2.0
b    4.0
c    6.0
d    8.0
e    NaN
dtype: float64

In [38]:
# Suma de dataframes
dataframe + dataframe1

,var1,var2,var3
a,2,b'11',NaN
b,4,b'22',NaN
c,6,b'33',NaN
d,8,b'44',NaN


In [39]:
# Producto de dataframes
dataframe * dataframe1

TypeError: can't multiply sequence by non-int of type 'bytes'

#### Trasposición - Sólo DataFrames

Podemos trasponer filas por columnas, pero únicamente en DataFrame (ya que las series sólo pueden ser unidimensionales). Básicamente lo que se realizará es intercambiar el índice de columnas por el de filas.

In [40]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [41]:
dataframe.T

,a,b,c,d
var1,1,2,3,4
var2,b'1',b'2',b'3',b'4'


#### Funciones de numpy (Universal functions, operaciones matemáticas...)

Podemos aplicar cualquier función de NumPy a cualquier estructura de pandas.<br/>

<b>IMPORTANTE:</b> De nuevo, al poder tener múltiples tipos en DataFrames no siempre se podrán aplicar las operaciones (o el resultado obtenido no será el esperado). Además, en el caso de DataFrames en caso de no indicar un valor para <b>axis</b> se aplicará la operación por columnas y nunca sobre el DataFrame completo.

In [45]:
dataframe

,var1,var2
a,1,b'1'
b,2,b'2'
c,3,b'3'
d,4,b'4'


In [42]:
# Operaciones sobre series
np.sqrt(serie)

a    1.000000
b    1.414214
c    1.732051
d    2.000000
dtype: float64

In [43]:
# Operaciones sobre dataframes (columna a columna, por defecto)
np.sum(dataframe)

var1         10
var2    b'1234'
dtype: object

In [46]:
# Operaciones sobre dataframes (especificando eje) RARO axis = 1
np.sum(dataframe, axis=1)

a    1
b    2
c    3
d    4
dtype: int64

## Indexación y slicing en pandas

Además de poder reutilizar los métodos de indexación y slicing de NumPy sobre Series y DataFrames (con las limitaciones ya comentadas), pandas pone a nuestra disposición nuevos métodos de indexación que permiten tener un mayor control sobre la misma y superar las limitaciones que nos impone NumPy sobre este tipo de estructuras. Veamos todas las posibles combinaciones.<br/>

#### Indexación por atributo de clave

Podemos indexar un elemento concreto de una Serie o una columna concreta de un DataFrame mediante el uso de su etiqueta/clave como atributo, con sintaxis obj.etiqueta.

In [47]:
serie = pd.Series([1, 2, 3, 4], index = ['a', 'b', 'c', 'd'])
serie.a

1

In [48]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4), index=['f1', 'f2', 'f3', 'f4'], columns=['c1','c2','c3','c4'])
dataframe.c1

f1     0
f2     4
f3     8
f4    12
Name: c1, dtype: int32

#### Indexación con sintáxis [] directa

<table>
<tr>
<th>Tipo</th>
<th>En Series</th>
<th>En DataFrames</th>
</tr>
<tr>
<td>obj[num_val]</td>
<td>Selección por posición (salvo si el índice es numérico)</td>
<td>Selección por clave de columna</td>
</tr>
<tr>
<td>obj[key]</td>
<td>Selección por clave</td>
<td>Selección por clave de columna</td>
</tr>
<tr>
<td>obj[num_val1:num_val2]</td>
<td>Selección por posición (salvo si el índice es numérico)</td>
<td>Selección por posición de fila (salvo si el índice de filas es numérico)</td>
</tr>
<tr>
<td>obj[key1:key2]</td>
<td>Selección por clave</td>
<td>Selección por clave de fila</td>
</tr>
<tr>
<td>obj[[num_val1,..,num_valn]]</td>
<td>Selección por posición (salvo si el índice es numérico)</td>
<td>Selección por posición de columna</td>
</tr>
<tr>
<td>obj[[key1,..,keyn]]</td>
<td>Selección por clave</td>
<td>Selección por clave de columna</td>
</tr>
<tr>
<td>obj[condition]</td>
<td>Selección por estructura booleana</td>
<td>Selección por estructura booleana</td>
</tr>
</table>

In [52]:
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [49]:
dataframe[['c1', 'c3', 'c2']]

,c1,c3,c2
f1,0,2,1
f2,4,6,5
f3,8,10,9
f4,12,14,13


In [50]:
dataframe.loc[(dataframe.c1 > 2), ['c1', 'c3']]

,c1,c3
f2,4,6
f3,8,10
f4,12,14


#### Indexación con método .loc - Por claves

<table>
<tr>
<th>Tipo</th>
<th>En Series</th>
<th>En DataFrames</th>
</tr>
<tr>
<td>obj.loc[key]</td>
<td>Selección por clave</td>
<td>Selección por clave de filas</td>
</tr>
<tr>
<td>obj.loc[key1:key2]</td>
<td>Selección por clave</td>
<td>Selección por clave de filas</td>
</tr>
<tr>
<td>obj.loc[[key1,...,keyn]]</td>
<td>Selección por clave</td>
<td>Selección por clave de filas</td>
</tr>
<tr>
<td>obj.loc[condition]</td>
<td>Selección por estructura booleana</td>
<td>Selección por estructura booleana sobre filas</td>
</tr>
<tr>
<td>obj.loc[sel1, sel2]</td>
<td>ERROR</td>
<td>Selección por clave de fila (sel1) y columna (sel2). Selectores: clave, slice, secuencia o condición</td>
</tr>
</table>

In [53]:
dataframe.loc['f1', 'c1']

0

In [54]:
dataframe.loc[:, 'c1']

f1     0
f2     4
f3     8
f4    12
Name: c1, dtype: int32

#### Indexación con método .iloc - Por índices

<table>
<tr>
<th>Tipo</th>
<th>En Series</th>
<th>En DataFrames</th>
</tr>
<tr>
<td>obj.iloc[num_val]</td>
<td>Selección por posición</td>
<td>Selección por posición de filas</td>
</tr>
<tr>
<td>obj.iloc[num_val1:num_val2]</td>
<td>Selección por posición</td>
<td>Selección por posición de filas</td>
</tr>
<tr>
<td>obj.iloc[[num_val1,...,num_valn]]</td>
<td>Selección por posición</td>
<td>Selección por posición de filas</td>
</tr>
<tr>
<td>obj.iloc[sel1, sel2]</td>
<td>ERROR</td>
<td>Selección por posición de fila (sel1) y columna (sel2). Selectores: posición, slice o secuencia</td>
</tr>
</table>

In [56]:
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [55]:
dataframe.iloc[0]

c1    0
c2    1
c3    2
c4    3
Name: f1, dtype: int32

In [57]:
dataframe.iloc[0:2].loc[:,['c2', 'c3']]

,c2,c3
f1,1,2
f2,5,6


In [58]:
dataframe[['c2', 'c3']].iloc[0:2]

,c2,c3
f1,1,2
f2,5,6


#### Indexación con método .ix - Mezcla de índices y claves

<b>IMPORTANTE:</b> Este método de indexación está desaconsejado porque ha sido marcado como DEPRECATED y desaparecerá en próximas versiones de pandas.

<table>
<tr>
<th>Tipo</th>
<th>En Series</th>
<th>En DataFrames</th>
</tr>
<tr>
<td>obj.ix[sel]</td>
<td>Selección por posición o clave</td>
<td>Selección por posición o clave de filas</td>
</tr>
<tr>
<td>obj.ix[sel1:sel2]</td>
<td>Selección por posición o clave</td>
<td>Selección por posición o clave de filas</td>
</tr>
<tr>
<td>obj.ix[[sel1,...,seln]]</td>
<td>Selección por posición o clave</td>
<td>Selección por posición o clave de filas</td>
</tr>
<tr>
<td>obj.ix[sel1, sel2]</td>
<td>Incorrecto (pero no ERROR)</td>
<td>Selección por posición o clave de fila (sel1) y columna (sel2). Selectores: posición o clave, slice, secuencia o condición</td>
</tr>
</table>

In [59]:
dataframe.ix[0:2, 'c1']

AttributeError: 'DataFrame' object has no attribute 'ix'

Al igual que en el core de Python y NumPy, un slice o selección se puede utilizar en el lado izquierdo de una asignación para modificar el contenido de los elementos dentro de la selección.

In [60]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [61]:
serie.iloc[0] = 100
serie

a    100
b      2
c      3
d      4
dtype: int64

In [62]:
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [64]:
dataframe.iloc[1:3, 2] = 100
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,100,7
f3,8,9,100,11
f4,12,13,14,15


In [67]:
dataframe[dataframe['c3'] > 15]

,c1,c2,c3,c4
f2,4,5,100,7
f3,8,9,100,11


In [66]:
dataframe[dataframe['c3'] > 15]['c1']

f2    4
f3    8
Name: c1, dtype: int32

In [68]:
filtro = [True, False, True, False]
dataframe[filtro]

,c1,c2,c3,c4
f1,0,1,2,3
f3,8,9,100,11


## Reindexación, establecimiento y descarte de índices

Pueden existir ocasiones en las que se desee modificar el índice de una estructura tras haberla creado. En este caso no se trata de cambiar las etiquetas asignadas sino de reordenaciones, eliminación o adición de etiquetas. Para ello, pandas nos ofrece el método <b>reindex</b>. Lo que obtendremos será una nueva estructura (copia) con el índice seleccionado. <br/><br/>
En el caso de que con el nuevo índice, los elementos nuevos se rellenarán a NaN. Para evitarlo, disponemos de los siguientes parámetros:<br/>
<ul>
<li><b>fill_value:</b> Relleno a un valor fijo establecido.</li>
<li><b>method:</b> Relleno según un método definido.
<ul>
<li>ffill: Relleno mediante la observación de los últimos valores rellenos.</li>
<li>bfill: Relleno mediante la observación de los próximos valores rellenos.</li>
<li>nearest: Relleno mediante la observación del valor más próximo.</li>
</ul>
</li>
</ul>

In [69]:
serie = pd.Series([1, 2, 3, 4], index=['a','b','c','d'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [70]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4), index=['f1','f2','f3','f4'], columns=['c1','c2','c3','c4'])
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [71]:
# Reordenación del índice de una Serie
serie.reindex(['d', 'a', 'b','c'])

d    4
a    1
b    2
c    3
dtype: int64

In [72]:
# Adición de etiquetas a un índice de una serie
serie.reindex(['a', 'b', 'e', 'd', 'c'])

a    1.0
b    2.0
e    NaN
d    4.0
c    3.0
dtype: float64

In [73]:
# Adición de etiquetas con valor de relleno a un índice de una serie
serie.reindex(['a', 'b', 'e', 'd', 'c'], fill_value=0)       

a    1
b    2
e    0
d    4
c    3
dtype: int64

In [76]:
serie.reindex(['a', 'b', 'e', 'd', 'c'], method='ffill')    #coge orden alfabetico de los index

a    1
b    2
e    4
d    4
c    3
dtype: int64

In [77]:
# Selección de índice de filas de un DataFrame
dataframe.reindex(['f1', 'f3'])

,c1,c2,c3,c4
f1,0,1,2,3
f3,8,9,10,11


In [78]:
# Adición de etiquetas al índice de filas de un DataFrame
dataframe.reindex(['f1', 'f3', 'f6'])

,c1,c2,c3,c4
f1,0.0,1.0,2.0,3.0
f3,8.0,9.0,10.0,11.0
f6,NaN,NaN,NaN,NaN


In [79]:
# Adición de etiquetas al índice de filas de un DataFrame con método de relleno
dataframe.reindex(['f1', 'f3', 'f10'], method='ffill')

,c1,c2,c3,c4
f1,0,1,2,3
f3,8,9,10,11
f10,0,1,2,3


In [80]:
# Modificación del índice de columnas de un DataFrame
dataframe.reindex(columns=['c1', 'c3', 'c2'])

,c1,c3,c2
f1,0,2,1
f2,4,6,5
f3,8,10,9
f4,12,14,13


En cualquier momento, podemos descartar el índice de un DataFrame incorporando el mismo como una columna más de nuestros datos. Esto lo haremos mediante la función <b>reset_index</b>. Esto hará que el índice pase a ser una secuencia numérica.

In [83]:
dataframe.reset_index()  #con drop=True no guarda la columna index dataframe.reset_index(drop=True) 

,index,c1,c2,c3,c4
0,f1,0,1,2,3
1,f2,4,5,6,7
2,f3,8,9,10,11
3,f4,12,13,14,15


Del mismo modo, podemos reestablecer un conjunto de columnas como índice de un DataFrame con la función <b>set_index</b>.

In [84]:
dataframe.set_index(['c1'])

,c2,c3,c4
c1,,,
0,1,2,3
4,5,6,7
8,9,10,11
12,13,14,15


## Tablas pivote

Siguiendo con las funciones de gestión de índices, pandas incluye la posibilidad de gestionar los mismos como si de una Pivot Table de Excel se tratase, haciendo mucho más sencillo el análisis de información resultante.

In [85]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013, 2001], 
             'Valoración':[6, None, 8.75, None, 8.9],
             'Presupuesto':[160, 250, 100, None, 93],
             'Director':['Peter Jackson', 'Gareth Edwards', 'Martin Scorsese', 'Alfonso Cuarón', 'Peter Jackson'],
             'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity', 'Lord of the Rings']}
)
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Peter Jackson,Godzilla
1,2014,NaN,250.0,Gareth Edwards,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2001,8.90,93.0,Peter Jackson,Lord of the Rings


In [86]:
# Filas: Año, Columnas: Director, Valores: Título
peliculas.pivot('Año', 'Director', 'Título')

Director,Alfonso Cuarón,Gareth Edwards,Martin Scorsese,Peter Jackson
Año,,,,
2001,NaN,NaN,NaN,Lord of the Rings
2013,Gravity,NaN,El lobo de Wall Street,NaN
2014,NaN,El Hobbit III,NaN,Godzilla


También podemos crear tablas pivote utilizando una función de agregación para los valores, de forma que se haga una agrupación de resultados.

In [87]:
# Hacemos que haya dos películas para el mismo año y director
peliculas.loc[peliculas['Año'] == 2001, 'Año'] = 2014
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Peter Jackson,Godzilla
1,2014,NaN,250.0,Gareth Edwards,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2014,8.90,93.0,Peter Jackson,Lord of the Rings


In [88]:
# Utilizamos la función pivot_table para establecer: valor, índice, columnas y función de agregación en caso de colisión
pd.pivot_table(peliculas, values='Presupuesto', index=['Director'], columns=['Año'], aggfunc=np.sum)

Año,2013,2014
Director,,
Alfonso Cuarón,0.0,NaN
Gareth Edwards,NaN,250.0
Martin Scorsese,100.0,NaN
Peter Jackson,NaN,253.0


## Eliminación de filas y/o columnas en pandas

Aunque el proceso de eliminación de columnas se puede hacer mediante la aplicación de los mismos métodos que en el caso de diccionarios, pandas pone a nuestra disposición el método <b>drop</b>.

In [89]:
serie

a    1
b    2
c    3
d    4
dtype: int64

In [90]:
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [91]:
# Eliminación de valores de una Serie
serie.drop('a')

b    2
c    3
d    4
dtype: int64

In [92]:
serie.drop(['a','d'])

b    2
c    3
dtype: int64

In [93]:
# Eliminación de filas de un DataFrame
dataframe.drop('f1')

,c1,c2,c3,c4
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [94]:
# Eliminación de columnas de un DataFrame
dataframe.drop('c2', axis=1)

,c1,c3,c4
f1,0,2,3
f2,4,6,7
f3,8,10,11
f4,12,14,15


## Aritmética con estructuras de pandas

Aunque, como ya se ha visto, podemos aprovechar la compatibilidad con NumPy para llevar a cabo operaciones aritméticas básicas, estas operaciones aplican el proceso de "alineación" de índices introduciendo valores NaN en los resultados cuando no hay coincidencia de claves. Para solucionar este problema, pandas nos ofrece algunas funciones de utilidad para las más básicas (suma, resta, multiplicación y división) que permiten establecer un valor de "relleno" en el caso de claves no coincidentes.

In [95]:
serie1 = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie1

a    1
b    2
c    3
d    4
dtype: int64

In [96]:
serie2 = pd.Series([5, 6, 7, 8], index=['c', 'd', 'e', 'f'])
serie2

c    5
d    6
e    7
f    8
dtype: int64

In [97]:
# Resultado de operación básica"
serie1 + serie2

a     NaN
b     NaN
c     8.0
d    10.0
e     NaN
f     NaN
dtype: float64

In [98]:
# Resultado con operación pandas
serie1.add(serie2)

a     NaN
b     NaN
c     8.0
d    10.0
e     NaN
f     NaN
dtype: float64

In [99]:
# Resultado con operación pandas y relleno
serie1.add(serie2, fill_value=0)

a     1.0
b     2.0
c     8.0
d    10.0
e     7.0
f     8.0
dtype: float64

In [100]:
serie1.sub(serie2, fill_value=0)

a    1.0
b    2.0
c   -2.0
d   -2.0
e   -7.0
f   -8.0
dtype: float64

In [101]:
serie1.mul(serie2, fill_value=0)

a     0.0
b     0.0
c    15.0
d    24.0
e     0.0
f     0.0
dtype: float64

In [102]:
serie1.div(serie2, fill_value=0)

a         inf
b         inf
c    0.600000
d    0.666667
e    0.000000
f    0.000000
dtype: float64

## Ordenación en estructuras de pandas

pandas pone a nuestra disposición varias formas de realizar ordenaciones de los contenidos de una Serie o un DataFrame. Vamos a ver los más utilizados.

#### Ordenación en Series

In [103]:
serie = pd.Series([3, 2, 1, 4], index=['d', 'a', 'c', 'b'])
serie

d    3
a    2
c    1
b    4
dtype: int64

In [104]:
# Ordenación por índice
serie.sort_index()

a    2
b    4
c    1
d    3
dtype: int64

In [105]:
# Ordenación descendente por índice
serie.sort_index(ascending=False)

d    3
c    1
b    4
a    2
dtype: int64

In [106]:
# Ordenación por valores
serie.sort_values()

c    1
a    2
d    3
b    4
dtype: int64

In [107]:
# Ordenación por valores descendente
serie.sort_values(ascending=False)

b    4
d    3
a    2
c    1
dtype: int64

#### Ordenación en DataFrames

In [108]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4), index=['f3', 'f1', 'f4', 'f2'], columns=['c3', 'c1', 'c4', 'c2'])
dataframe

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [109]:
# Ordenación por índice de filas
dataframe.sort_index()

,c3,c1,c4,c2
f1,4,5,6,7
f2,12,13,14,15
f3,0,1,2,3
f4,8,9,10,11


In [110]:
# Ordenación por índice de columnas
dataframe.sort_index(axis=1)

,c1,c2,c3,c4
f3,1,3,0,2
f1,5,7,4,6
f4,9,11,8,10
f2,13,15,12,14


In [111]:
# Ordenación descendente por índice de filas
dataframe.sort_index(ascending=False)

,c3,c1,c4,c2
f4,8,9,10,11
f3,0,1,2,3
f2,12,13,14,15
f1,4,5,6,7


In [112]:
# Ordenación por valores de filas
dataframe.sort_values('f1', axis=1)

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [113]:
dataframe.sort_values('f1', axis=1, ascending=False)

,c2,c4,c1,c3
f3,3,2,1,0
f1,7,6,5,4
f4,11,10,9,8
f2,15,14,13,12


In [114]:
dataframe.sort_values(['f1', 'f2'], axis=1)

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [115]:
dataframe.sort_values(['f1', 'f2'], axis=1, ascending=[False, True])

,c2,c4,c1,c3
f3,3,2,1,0
f1,7,6,5,4
f4,11,10,9,8
f2,15,14,13,12


In [116]:
# Ordenación por valores de columnas
dataframe.sort_values(['c1'])

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [118]:
dataframe.reset_index(drop=True).sort_values(['c1'], ascending=False).reset_index(drop=True)

,c3,c1,c4,c2
0,12,13,14,15
1,8,9,10,11
2,4,5,6,7
3,0,1,2,3


## Recuperación de muestras parciales del contenido

En estructuras de datos potencialmente grandes, suele ser muy necesaria la recuperación de una muestra de ejemplo de un conjunto reducido de elementos que permitan hacerse una idea del contenido de la estructura sin necesidad de listar TODO el contenido de la misma. Pandas, como R, pone a nuestra disposición dos métodos <b>head</b> (para obtener un muestra del inicio de la estructura) y <b>tail</b> para obtener la muestras del final. Ambos métodos recibirán como parámetro el número de registros a recuperar.

In [119]:
serie = pd.Series(np.arange(100))
serie

0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Length: 100, dtype: int32

In [120]:
dataframe = pd.DataFrame(np.arange(100).reshape(10, 10))
dataframe

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,10,11,12,13,14,15,16,17,18,19
2,20,21,22,23,24,25,26,27,28,29
3,30,31,32,33,34,35,36,37,38,39
4,40,41,42,43,44,45,46,47,48,49
5,50,51,52,53,54,55,56,57,58,59
6,60,61,62,63,64,65,66,67,68,69
7,70,71,72,73,74,75,76,77,78,79
8,80,81,82,83,84,85,86,87,88,89
9,90,91,92,93,94,95,96,97,98,99


In [121]:
# Recuperación de los 5 primeros elementos de una Serie
serie.head()

0    0
1    1
2    2
3    3
4    4
dtype: int32

In [122]:
# Recueperación de los 5 últimos elementos de una Serie
serie.tail()

95    95
96    96
97    97
98    98
99    99
dtype: int32

In [123]:
# Recuperación de los 3 primeros elementos de un DataFrame
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,10,11,12,13,14,15,16,17,18,19
2,20,21,22,23,24,25,26,27,28,29


In [124]:
# Recueperación de los 3 últimos elementos de un DataFrame
dataframe.tail(3)

,0,1,2,3,4,5,6,7,8,9
7,70,71,72,73,74,75,76,77,78,79
8,80,81,82,83,84,85,86,87,88,89
9,90,91,92,93,94,95,96,97,98,99
